# 写在前面

最近一周在清华装Kubernetes，真的是一把辛酸泪，前几天都快绝望了，还好在csdn上找到一篇博客详细地介绍了安装Kubernetes的相关事宜，我在这篇文章的指导下才渐渐将一个双节点的Kubernets集群在阿里云上搭建起来，虽然教程里面也有说的不清楚的地方，但是还是给了我从无到有的启发，现在就把安装的细节写下来，以供参考。

# 安装环境

清华提供了一个`6`节点的集群，计划搭建一个6节点的Kubernetes集群，各个节点的配置如下所示。


|IP地址|操作系统|角色|
|-------|----------|--------------|
|192.168.10.10|Ubuntu 16.04.3 LTS|Master|
|192.168.10.11|Ubuntu 16.04.3 LTS|Slave1|
|192.168.10.12|Ubuntu 16.04.3 LTS|Slave2|
|192.168.10.13|Ubuntu 16.04.3 LTS|Slave3|
|192.168.10.14|Ubuntu 16.04.3 LTS|Slave4|
|192.168.10.15|Ubuntu 16.04.3 LTS|Slave5|

把主节点的`/etc/hosts`文件展示如下：

```shell
zdyfjh@Master:~$ cat /etc/hosts
127.0.0.1 localhost

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
101.6.6.177 mirrors.tuna.tsinghua.edu.cn
192.168.10.10 Master
192.168.10.11 Slave1
192.168.10.12 Slave2
192.168.10.13 Slave3
192.168.10.14 Slave4
192.168.10.15 Slave5
151.101.24.175 bootstrap.pypa.io
151.101.26.49 curl.haxx.se
```

# 组件下载

由于装系统这种事，有一定的玄学成分，常常是同一个东西A版本就很正常，B版本就呵呵了，因此为了保证安装的正确性，将用到的组件下载地址在这里给出，推荐下载这里给出的组件。

1. Etcd: [https://github.com/etcd-io/etcd/releases/download/v3.3.10/etcd-v3.3.10-linux-amd64.tar.gz](https://github.com/etcd-io/etcd/releases/download/v3.3.10/etcd-v3.3.10-linux-amd64.tar.gz)

2. Kubernetes: [https://dl.k8s.io/v1.9.6/kubernetes-server-linux-amd64.tar.gz](https://dl.k8s.io/v1.9.6/kubernetes-server-linux-amd64.tar.gz)

3. Flannel: [https://github.com/coreos/flannel/releases/download/v0.9.0/flannel-v0.9.0-linux-amd64.tar.gz](https://github.com/coreos/flannel/releases/download/v0.9.0/flannel-v0.9.0-linux-amd64.tar.gz)

# 配置CA证书

该步骤主要是为了使集群上能够使用`https`协议，这里使用 CloudFlare 的 PKI 工具集 cfssl 来生成 Certificate Authority (CA) 证书和秘钥文件。

## Master上操作

* ### 安装`cfssl`

```shell
root@Master:/home/zdyfjh# mkdir -p /opt/local/cfssl
root@Master:/home/zdyfjh# cd /opt/local/cfssl
root@Master:/opt/local/cfssl# wget https://pkg.cfssl.org/R1.2/cfssl_linux-amd64
root@Master:/opt/local/cfssl# mv cfssl_linux-amd64 cfssl
root@Master:/opt/local/cfssl# wget https://pkg.cfssl.org/R1.2/cfssljson_linux-amd64
root@Master:/opt/local/cfssl# mv cfssljson_linux-amd64 cfssljson
root@Master:/opt/local/cfssl# wget https://pkg.cfssl.org/R1.2/cfssl-certinfo_linux-amd64
root@Master:/opt/local/cfssl# mv cfssl-certinfo_linux-amd64 cfssl-certinfo
root@Master:/opt/local/cfssl# chmod +x *
root@Master:/opt/local/cfssl# ll
total 18816
drwxr-xr-x 2 root root     4096 Oct 22 21:34 ./
drwxr-xr-x 3 root root     4096 Oct 22 21:30 ../
-rwxr-xr-x 1 root root 10376657 Mar 30  2016 cfssl*
-rwxr-xr-x 1 root root  6595195 Mar 30  2016 cfssl-certinfo*
-rwxr-xr-x 1 root root  2277873 Mar 30  2016 cfssljson*
```

* ### 创建CA证书配置文件

```shell
root@Master:/opt/local/cfssl# mkdir /opt/ssl
root@Master:/opt/local/cfssl# cd /opt/ssl
root@Master:/opt/ssl# vim config.json
root@Master:/opt/ssl# cat config.json 
{
  "signing": {
    "default": {
      "expiry": "87600h"
    },
    "profiles": {
      "kubernetes": {
        "usages": [
            "signing",
            "key encipherment",
            "server auth",
            "client auth"
        ],
        "expiry": "87600h"
      }
    }
  }
}

root@Master:/opt/ssl# vim csr.json
root@Master:/opt/ssl# cat csr.json 
{
  "CN": "kubernetes",
  "key": {
    "algo": "rsa",
    "size": 2048
  },
  "names": [
    {
      "C": "CN",
      "ST": "Beijing",
      "L": "Beijing",
      "O": "k8s",
      "OU": "System"
    }
  ]
}
root@Master:/opt/ssl# ls
config.json  csr.json
root@Master:/opt/ssl# /opt/local/cfssl/cfssl gencert -initca csr.json | /opt/local/cfssl/cfssljson -bare ca
2018/10/22 21:43:35 [INFO] generating a new CA key and certificate from CSR
2018/10/22 21:43:35 [INFO] generate received request
2018/10/22 21:43:35 [INFO] received CSR
2018/10/22 21:43:35 [INFO] generating key: rsa-2048
2018/10/22 21:43:36 [INFO] encoded CSR
2018/10/22 21:43:36 [INFO] signed certificate with serial number 52404311838458003012572611311024350631102884515
root@Master:/opt/ssl# ls -l
total 20
-rw-r--r-- 1 root root 1001 Oct 22 21:43 ca.csr
-rw------- 1 root root 1675 Oct 22 21:43 ca-key.pem
-rw-r--r-- 1 root root 1359 Oct 22 21:43 ca.pem
-rw-r--r-- 1 root root  292 Oct 22 21:37 config.json
-rw-r--r-- 1 root root  208 Oct 22 21:39 csr.json
```

* ### 向主节点及从节点分发证书（依然只在主节点上操作）

```shell
root@Master:/opt/ssl# mkdir -p /etc/kubernetes/ssl
root@Master:/opt/ssl# cp *.pem /etc/kubernetes/ssl
root@Master:/opt/ssl# cp ca.csr /etc/kubernetes/ssl
```
**注意：一下操作要通过`scp`命令向各个从节点发送证书，证书存放的路径都为`/etc/kubernetes/ss/`，因此要确保各个从节点上均存在这个目录**

```shell
root@Master:/opt/ssl# scp * zdyfjh@192.168.10.11:/etc/kubernetes/ssl
root@Master:/opt/ssl# scp * zdyfjh@192.168.10.12:/etc/kubernetes/ssl
root@Master:/opt/ssl# scp * zdyfjh@192.168.10.13:/etc/kubernetes/ssl
root@Master:/opt/ssl# scp * zdyfjh@192.168.10.14:/etc/kubernetes/ssl
root@Master:/opt/ssl# scp * zdyfjh@192.168.10.15:/etc/kubernetes/ssl
```

# 安装Docker（该步骤各个从节点也要进行）

## Master节点上操作

```shell
root@Master:/opt/ssl# apt-get update
root@Master:/opt/ssl# apt-get remove docker docker-engine docker.io
root@Master:/opt/ssl# apt-get install docker.io
root@Master:/opt/ssl# docker -v
Docker version 17.03.2-ce, build f5ec1e2
```

## Slave节点上操作

```shell
root@Slave:/opt/ssl# apt-get update
root@Slave:/opt/ssl# apt-get remove docker docker-engine docker.io
root@Slave:/opt/ssl# apt-get install docker.io
root@Slave:/opt/ssl# docker -v
Docker version 17.03.2-ce, build f5ec1e2
```

# 安装Etcd（该步骤需要从节点操作）

## Master节点上操作

* ### 下载Etcd源代码

```shell
root@Master:/home/zdyfjh/download# wget https://github.com/etcd-io/etcd/releases/download/v3.3.10/etcd-v3.3.10-linux-amd64.tar.gz
root@Master:/home/zdyfjh/download# ls
etcd-v3.3.10-linux-amd64.tar.gz
```

* ### 向各个从节点分发Etcd源代码（**注意：各个从节点上要建立好相应的目录**）

```shell
root@Master:/home/zdyfjh/download# scp etcd-v3.3.10-linux-amd64.tar.gz zdyfjh@192.168.10.11:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp etcd-v3.3.10-linux-amd64.tar.gz zdyfjh@192.168.10.12:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp etcd-v3.3.10-linux-amd64.tar.gz zdyfjh@192.168.10.13:/home/zdyfjh/download 
root@Master:/home/zdyfjh/download# scp etcd-v3.3.10-linux-amd64.tar.gz zdyfjh@192.168.10.14:/home/zdyfjh/download    
root@Master:/home/zdyfjh/download# scp etcd-v3.3.10-linux-amd64.tar.gz zdyfjh@192.168.10.15:/home/zdyfjh/download
```

* ### 安装Etcd

```shell
root@Master:/home/zdyfjh/download# tar -zxvf etcd-v3.3.10-linux-amd64.tar.gz
root@Master:/home/zdyfjh/download# cd ./etcd-v3.3.10-linux-amd64
root@Master:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# ls
Documentation  etcd  etcdctl  README-etcdctl.md  README.md  READMEv2-etcdctl.md
root@Master:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# mv etcd* /usr/bin/
root@Master:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# ll /usr/bin/ | grep etcd
-rwxr-xr-x  1 6810230 users   19237536 Oct 11 01:32 etcd*
-rwxr-xr-x  1 6810230 users   15817472 Oct 11 01:32 etcdctl*
```

* ### 为Etcd配置秘钥（证书）并创建数据目录

```shell
root@Master:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# cd /opt/ssl/
root@Master:/opt/ssl# vim etcd-csr.json
root@Master:/opt/ssl# cat etcd-csr.json 
{
  "CN": "etcd",
  "hosts": [
    "192.168.10.10",
    "192.168.10.11",
    "192.168.10.12",
    "192.168.10.13",
    "192.168.10.14",
    "192.168.10.15"
  ],
  "key": {
    "algo": "rsa",
    "size": 2048
  },
  "names": [
    {
      "C": "CN",
      "ST": "Beijing",
      "L": "Beijing",
      "O": "k8s",
      "OU": "System"
    }
  ]
}
root@Master:/opt/ssl# /opt/local/cfssl/cfssl gencert -ca=/opt/ssl/ca.pem \
>   -ca-key=/opt/ssl/ca-key.pem \
>   -config=/opt/ssl/config.json \
>   -profile=kubernetes etcd-csr.json | /opt/local/cfssl/cfssljson -bare etcd
2018/10/23 10:15:19 [INFO] generate received request
2018/10/23 10:15:19 [INFO] received CSR
2018/10/23 10:15:19 [INFO] generating key: rsa-2048
2018/10/23 10:15:20 [INFO] encoded CSR
2018/10/23 10:15:20 [INFO] signed certificate with serial number 160692721133621926504549778955227969855580110303
2018/10/23 10:15:20 [WARNING] This certificate lacks a "hosts" field. This makes it unsuitable for
websites. For more information see the Baseline Requirements for the Issuance and Management
of Publicly-Trusted Certificates, v.1.1.6, from the CA/Browser Forum (https://cabforum.org);
specifically, section 10.2.3 ("Information Requirements").
root@Master:/opt/ssl# ls etcd*
etcd.csr  etcd-csr.json  etcd-key.pem  etcd.pem
root@Master:/opt/ssl# cp etcd*.pem /etc/kubernetes/ssl/
root@Master:/opt/ssl# scp etcd*.pem zdyfjh@192.168.10.11:/etc/kubernetes/ssl/ 
root@Master:/opt/ssl# scp etcd*.pem zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp etcd*.pem zdyfjh@192.168.10.13:/etc/kubernetes/ssl/    
root@Master:/opt/ssl# scp etcd*.pem zdyfjh@192.168.10.14:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp etcd*.pem zdyfjh@192.168.10.15:/etc/kubernetes/ssl/
root@Master:/opt/ssl# chmod 644 /etc/kubernetes/ssl/etcd-key.pem
root@Master:/home/zdyfjh# useradd etcd
root@Master:/home/zdyfjh# mkdir -p /opt/etcd
root@Master:/home/zdyfjh# chown -R etcd:etcd /opt/etcd             
```

* ### 创建service文件

```shell
root@Master:/opt/ssl# vim /etc/systemd/system/etcd.service
root@Master:/opt/ssl# cat /etc/systemd/system/etcd.service 
[Unit]
Description=Etcd Server
After=network.target
After=network-online.target
Wants=network-online.target

[Service]
Type=notify
WorkingDirectory=/opt/etcd/
User=etcd
# set GOMAXPROCS to number of processors
ExecStart=/usr/bin/etcd \
  --name=etcd1 \ # 注意：这里要根据实际填入etcd1，etcd2等名称，具体的填入内容参照--initial-cluster中的内容名
  --cert-file=/etc/kubernetes/ssl/etcd.pem \
  --key-file=/etc/kubernetes/ssl/etcd-key.pem \
  --peer-cert-file=/etc/kubernetes/ssl/etcd.pem \
  --peer-key-file=/etc/kubernetes/ssl/etcd-key.pem \
  --trusted-ca-file=/etc/kubernetes/ssl/ca.pem \
  --peer-trusted-ca-file=/etc/kubernetes/ssl/ca.pem \
  --initial-advertise-peer-urls=https://192.168.10.10:2380 \ #这里的ip要改成实际机器的IP
  --listen-peer-urls=https://192.168.10.10:2380 \ #这里的ip要改成实际机器的IP
  --listen-client-urls=https://192.168.10.10:2379,http://127.0.0.1:2379 \ #这里的ip要改成实际机器的IP
  --advertise-client-urls=https://192.168.10.10:2379 \ #这里的ip要改成实际机器的IP
  --initial-cluster-token=k8s-etcd-cluster \
  --initial-cluster=etcd1=https://192.168.10.10:2380,etcd2=https://192.168.10.11:2380,etcd3=https://192.168.10.12:2380,etcd4=https://192.168.10.13:2380,etcd5=https://192.168.10.14:2380,etcd6=https://192.168.10.15:2380 \
  --initial-cluster-state=new \
  --data-dir=/opt/etcd/
Restart=on-failure
RestartSec=5
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
```

## Slave节点的操作

* ### 安装Etcd

```shell
root@Slave:/home/zdyfjh/download# tar -zxvf etcd-v3.3.10-linux-amd64.tar.gz
root@Slave:/home/zdyfjh/download# cd ./etcd-v3.3.10-linux-amd64
root@Slave:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# ls
Documentation  etcd  etcdctl  README-etcdctl.md  README.md  READMEv2-etcdctl.md
root@Slave:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# mv etcd* /usr/bin/
root@Slave:/home/zdyfjh/download/etcd-v3.3.10-linux-amd64# ll /usr/bin/ | grep etcd
-rwxr-xr-x  1 6810230 users   19237536 Oct 11 01:32 etcd*
-rwxr-xr-x  1 6810230 users   15817472 Oct 11 01:32 etcdctl*
```

* ### 更改权限并创建数据目录

```shell
root@Slave:/home/zdyfjh# chmod 644 /etc/kubernetes/ssl/etcd-key.pem
root@Slave:/home/zdyfjh# useradd etcd
root@Slave:/home/zdyfjh# mkdir -p /opt/etcd
root@Slave:/home/zdyfjh# chown -R etcd:etcd /opt/etcd
```

* ### 创建service文件

```shell
root@Slave:/opt/ssl# vim /etc/systemd/system/etcd.service
root@Slave:/opt/ssl# cat /etc/systemd/system/etcd.service 
[Unit]
Description=Etcd Server
After=network.target
After=network-online.target
Wants=network-online.target

[Service]
Type=notify
WorkingDirectory=/opt/etcd/
User=etcd
# set GOMAXPROCS to number of processors
ExecStart=/usr/bin/etcd \
  --name=etcd1 \ # 注意：这里要根据实际填入etcd1，etcd2等名称，具体的填入内容参照--initial-cluster中的内容名
  --cert-file=/etc/kubernetes/ssl/etcd.pem \
  --key-file=/etc/kubernetes/ssl/etcd-key.pem \
  --peer-cert-file=/etc/kubernetes/ssl/etcd.pem \
  --peer-key-file=/etc/kubernetes/ssl/etcd-key.pem \
  --trusted-ca-file=/etc/kubernetes/ssl/ca.pem \
  --peer-trusted-ca-file=/etc/kubernetes/ssl/ca.pem \
  --initial-advertise-peer-urls=https://192.168.10.10:2380 \ #这里的ip要改成实际机器的IP
  --listen-peer-urls=https://192.168.10.10:2380 \ #这里的ip要改成实际机器的IP
  --listen-client-urls=https://192.168.10.10:2379,http://127.0.0.1:2379 \ #这里的ip要改成实际机器的IP
  --advertise-client-urls=https://192.168.10.10:2379 \ #这里的ip要改成实际机器的IP
  --initial-cluster-token=k8s-etcd-cluster \
  --initial-cluster=etcd1=https://192.168.10.10:2380,etcd2=https://192.168.10.11:2380,etcd3=https://192.168.10.12:2380,etcd4=https://192.168.10.13:2380,etcd5=https://192.168.10.14:2380,etcd6=https://192.168.10.15:2380 \
  --initial-cluster-state=new \
  --data-dir=/opt/etcd/
Restart=on-failure
RestartSec=5
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
```

* ### 启动Etcd（所有节点上都要执行）

**注意：Etcd是一个分布式数据，因此一定要多个节点同时启动，如果间隔时间过长可能会出现错误**

```shell
root@Slave:/home/zdyfjh# systemctl daemon-reload
root@Slave:/home/zdyfjh# systemctl enable etcd
root@Slave:/home/zdyfjh# systemctl start etcd
```

## Master节点上的操作

* ### 启动Etcd（所有节点都要执行）

```shell
root@Master:/home/zdyfjh# systemctl daemon-reload
root@Master:/home/zdyfjh# systemctl enable etcd
root@Master:/home/zdyfjh# systemctl start etcd
root@Master:/home/zdyfjh# systemctl status etcd
● etcd.service - Etcd Server
   Loaded: loaded (/etc/systemd/system/etcd.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 11:12:02 CST; 13s ago
 Main PID: 27682 (etcd)
    Tasks: 48
   Memory: 31.2M
      CPU: 1.307s
   CGroup: /system.slice/etcd.service
           └─27682 /usr/bin/etcd --name=etcd1 --cert-file=/etc/kubernetes/ssl/etcd.pem --key-file=/etc/kubernetes/ssl/etcd-key.pem --peer-cer

Oct 23 11:12:06 Master etcd[27682]: health check for peer 5f9042fd5c3dd434 could not connect: dial tcp 192.168.10.11:2380: connect: connectio
Oct 23 11:12:07 Master etcd[27682]: peer 5f9042fd5c3dd434 became active
Oct 23 11:12:07 Master etcd[27682]: established a TCP streaming connection with peer 5f9042fd5c3dd434 (stream MsgApp v2 reader)
Oct 23 11:12:07 Master etcd[27682]: established a TCP streaming connection with peer 5f9042fd5c3dd434 (stream Message reader)
Oct 23 11:12:07 Master etcd[27682]: established a TCP streaming connection with peer 5f9042fd5c3dd434 (stream MsgApp v2 writer)
Oct 23 11:12:07 Master etcd[27682]: established a TCP streaming connection with peer 5f9042fd5c3dd434 (stream Message writer)
Oct 23 11:12:10 Master etcd[27682]: updated the cluster version from 3.0 to 3.3
Oct 23 11:12:10 Master etcd[27682]: enabled capabilities for version 3.3
Oct 23 11:12:11 Master etcd[27682]: health check for peer 5f9042fd5c3dd434 could not connect: dial tcp 192.168.10.11:2380: connect: connectio
Oct 23 11:12:11 Master etcd[27682]: health check for peer 5f9042fd5c3dd434 could not connect: dial tcp 192.168.10.11:2380: connect: connectio
```

* ### 查看Etcd的运行状态，同时鉴定证书可读性

```shell
root@Master:/home/zdyfjh# etcdctl --endpoints=https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379\
>         --cert-file=/etc/kubernetes/ssl/etcd.pem \
>         --ca-file=/etc/kubernetes/ssl/ca.pem \
>         --key-file=/etc/kubernetes/ssl/etcd-key.pem \
>         cluster-health
member ebec49f7cca334c is healthy: got healthy result from https://192.168.10.13:2379
member fe8d75804df3c84 is healthy: got healthy result from https://192.168.10.14:2379
member 2da67ab84f1e0230 is healthy: got healthy result from https://192.168.10.12:2379
member 4ae14a68da42d271 is healthy: got healthy result from https://192.168.10.10:2379
member 5f9042fd5c3dd434 is healthy: got healthy result from https://192.168.10.11:2379
member b81b08d39ad7e16f is healthy: got healthy result from https://192.168.10.15:2379
cluster is healthy
```

* ### 查看Etcd成员

```shell
root@Master:/home/zdyfjh# etcdctl --endpoints=https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379\
>         --cert-file=/etc/kubernetes/ssl/etcd.pem \
>         --ca-file=/etc/kubernetes/ssl/ca.pem \
>         --key-file=/etc/kubernetes/ssl/etcd-key.pem \
>         member list
ebec49f7cca334c: name=etcd4 peerURLs=https://192.168.10.13:2380 clientURLs=https://192.168.10.13:2379 isLeader=true
fe8d75804df3c84: name=etcd5 peerURLs=https://192.168.10.14:2380 clientURLs=https://192.168.10.14:2379 isLeader=false
2da67ab84f1e0230: name=etcd3 peerURLs=https://192.168.10.12:2380 clientURLs=https://192.168.10.12:2379 isLeader=false
4ae14a68da42d271: name=etcd1 peerURLs=https://192.168.10.10:2380 clientURLs=https://192.168.10.10:2379 isLeader=false
5f9042fd5c3dd434: name=etcd2 peerURLs=https://192.168.10.11:2380 clientURLs=https://192.168.10.11:2379 isLeader=false
b81b08d39ad7e16f: name=etcd6 peerURLs=https://192.168.10.15:2380 clientURLs=https://192.168.10.15:2379 isLeader=false
```

# 安装Kubernetes1.9.6

## Master上操作

* ### 下载源代码

```shell
root@Master:/home/zdyfjh/download# wget https://dl.k8s.io/v1.9.6/kubernetes-server-linux-amd64.tar.gz
root@Master:/home/zdyfjh/download# ls -a | grep kube
kubernetes-server-linux-amd64.tar.gz
root@Master:/home/zdyfjh/download# scp kubernetes-server-linux-amd64.tar.gz zdyfjh@192.168.10.11:/home/zdyfjh/download 
root@Master:/home/zdyfjh/download# scp kubernetes-server-linux-amd64.tar.gz zdyfjh@192.168.10.12:/home/zdyfjh/download
root@Master:/home/zdyfjh/download# scp kubernetes-server-linux-amd64.tar.gz zdyfjh@192.168.10.13:/home/zdyfjh/download
root@Master:/home/zdyfjh/download# scp kubernetes-server-linux-amd64.tar.gz zdyfjh@192.168.10.14:/home/zdyfjh/download
root@Master:/home/zdyfjh/download# scp kubernetes-server-linux-amd64.tar.gz zdyfjh@192.168.10.15:/home/zdyfjh/download
root@Master:/home/zdyfjh/download# tar -zxf kubernetes-server-linux-amd64.tar.gz ^C
root@Master:/home/zdyfjh/download# cd ./kubernetes
root@Master:/home/zdyfjh/download/kubernetes# cp -r server/bin/{kube-apiserver,kube-controller-manager,kube-scheduler,kubectl,kubelet,kube-proxy} /usr/local/bin/
root@Master:/home/zdyfjh/download/kubernetes# ls /usr/local/bin/ | grep kube
kube-apiserver
kube-controller-manager
kubectl
kubelet
kube-proxy
kube-scheduler
```

* ### 配置证书

```shell
root@Master:/home/zdyfjh/download/kubernetes# cd /opt/ssl/
root@Master:/opt/ssl# vim admin-csr.json
root@Master:/opt/ssl# cat admin-csr.json 
{
  "CN": "admin",
  "hosts": [],
  "key": {
    "algo": "rsa",
    "size": 2048
  },
  "names": [
    {
      "C": "CN",
      "ST": "Beijing",
      "L": "Beijing",
      "O": "system:masters",
      "OU": "System"
    }
  ]
}
root@Master:/opt/ssl# /opt/local/cfssl/cfssl gencert -ca=/etc/kubernetes/ssl/ca.pem \
>   -ca-key=/etc/kubernetes/ssl/ca-key.pem \
>   -config=/opt/ssl/config.json \
>   -profile=kubernetes admin-csr.json | /opt/local/cfssl/cfssljson -bare admin
2018/10/23 14:41:32 [INFO] generate received request
2018/10/23 14:41:32 [INFO] received CSR
2018/10/23 14:41:32 [INFO] generating key: rsa-2048
2018/10/23 14:41:32 [INFO] encoded CSR
2018/10/23 14:41:32 [INFO] signed certificate with serial number 695643913881607257811736010743672424384952693292
2018/10/23 14:41:32 [WARNING] This certificate lacks a "hosts" field. This makes it unsuitable for
websites. For more information see the Baseline Requirements for the Issuance and Management
of Publicly-Trusted Certificates, v.1.1.6, from the CA/Browser Forum (https://cabforum.org);
specifically, section 10.2.3 ("Information Requirements").
root@Master:/opt/ssl# ls admin*
admin.csr  admin-csr.json  admin-key.pem  admin.pem
root@Master:/opt/ssl# cp admin*.pem /etc/kubernetes/ssl/
root@Master:/opt/ssl# scp admin*.pem zdyfjh@192.168.10.11:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp admin*.pem zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp admin*.pem zdyfjh@192.168.10.13:/etc/kubernetes/ssl/  
root@Master:/opt/ssl# scp admin*.pem zdyfjh@192.168.10.14:/etc/kubernetes/ssl/ 
root@Master:/opt/ssl# scp admin*.pem zdyfjh@192.168.10.15:/etc/kubernetes/ssl/
```

* ### 配置Kubernetes集群

```shell
root@Master:/opt/ssl# kubectl config set-cluster kubernetes \
>   --certificate-authority=/etc/kubernetes/ssl/ca.pem \
>   --embed-certs=true \
>   --server=https://192.168.10.10:6443
Cluster "kubernetes" set.
root@Master:/opt/ssl# kubectl config set-credentials admin \
>   --client-certificate=/etc/kubernetes/ssl/admin.pem \
>   --embed-certs=true \
>   --client-key=/etc/kubernetes/ssl/admin-key.pem
User "admin" set.
root@Master:/opt/ssl# kubectl config set-context kubernetes \
>   --cluster=kubernetes \
>   --user=admin
Context "kubernetes" created.
root@Master:/opt/ssl# kubectl config use-context kubernetes
Switched to context "kubernetes".
root@Master:/opt/ssl# cd ~/.kube
root@Master:~/.kube# ls
config
root@Master:~/.kube# scp * zdyfjh@192.168.10.11:~/download    
root@Master:~/.kube# scp * zdyfjh@192.168.10.12:~/download    
root@Master:~/.kube# scp * zdyfjh@192.168.10.13:~/download  
root@Master:~/.kube# scp * zdyfjh@192.168.10.14:~/download   
root@Master:~/.kube# scp * zdyfjh@192.168.10.15:~/download
```

* ### 创建SSL证书

```shell
root@Master:~/.kube# cd /opt/ssl
root@Master:/opt/ssl# vim kubernetes-csr.json
root@Master:/opt/ssl# cat kubernetes-csr.json 
{
  "CN": "kubernetes",
  "hosts": [
    "192.168.10.10",
    "10.254.0.1",
    "kubernetes",
    "kubernetes.default",
    "kubernetes.default.svc",
    "kubernetes.default.svc.cluster",
    "kubernetes.default.svc.cluster.local"
  ],
  "key": {
    "algo": "rsa",
    "size": 2048
  },
  "names": [
    {
      "C": "CN",
      "ST": "Beijing",
      "L": "Beijing",
      "O": "k8s",
      "OU": "System"
    }
  ]
}
root@Master:/opt/ssl# /opt/local/cfssl/cfssl gencert -ca=/etc/kubernetes/ssl/ca.pem \
>   -ca-key=/etc/kubernetes/ssl/ca-key.pem \
>   -config=/opt/ssl/config.json \
>   -profile=kubernetes kubernetes-csr.json | /opt/local/cfssl/cfssljson -bare kubernetes
2018/10/23 15:21:03 [INFO] generate received request
2018/10/23 15:21:03 [INFO] received CSR
2018/10/23 15:21:03 [INFO] generating key: rsa-2048
2018/10/23 15:21:03 [INFO] encoded CSR
2018/10/23 15:21:03 [INFO] signed certificate with serial number 80697406592929186537048906789746052100450546141
2018/10/23 15:21:03 [WARNING] This certificate lacks a "hosts" field. This makes it unsuitable for
websites. For more information see the Baseline Requirements for the Issuance and Management
of Publicly-Trusted Certificates, v.1.1.6, from the CA/Browser Forum (https://cabforum.org);
specifically, section 10.2.3 ("Information Requirements").
root@Master:/opt/ssl# ls -l kubernetes*
-rw-r--r-- 1 root root 1236 Oct 23 15:21 kubernetes.csr
-rw-r--r-- 1 root root  420 Oct 23 15:20 kubernetes-csr.json
-rw------- 1 root root 1675 Oct 23 15:21 kubernetes-key.pem
-rw-r--r-- 1 root root 1602 Oct 23 15:21 kubernetes.pem
root@Master:/opt/ssl# cp kubernetes*.pem /etc/kubernetes/ssl/
root@Master:/opt/ssl# scp kubernetes*.pem zdyfjh@192.168.10.11:/etc/kubernetes/ssl/    
root@Master:/opt/ssl# scp kubernetes*.pem zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp kubernetes*.pem zdyfjh@192.168.10.13:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp kubernetes*.pem zdyfjh@192.168.10.14:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp kubernetes*.pem zdyfjh@192.168.10.15:/etc/kubernetes/ssl/
```

* ### 生成Token

```shell
root@Master:/opt/ssl# head -c 16 /dev/urandom | od -An -t x | tr -d ' '
f27f606aa0bb8bea34b1e22aabd3aab8
root@Master:/opt/ssl# vim token.csv
root@Master:/opt/ssl# cat token.csv 
f27f606aa0bb8bea34b1e22aabd3aab8,kubelet-bootstrap,10001,"system:kubelet-bootstrap"
root@Master:/opt/ssl# cp token.csv /etc/kubernetes/
root@Master:/opt/ssl# scp token.csv zdyfjh@192.168.10.11:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp token.csv zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp token.csv zdyfjh@192.168.10.13:/etc/kubernetes/ssl/    
root@Master:/opt/ssl# scp token.csv zdyfjh@192.168.10.14:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp token.csv zdyfjh@192.168.10.15:/etc/kubernetes/ssl/ 
```

* ### 生成高级审核文件

```shell
root@Master:/opt/ssl# cd /etc/kubernetes
root@Master:/etc/kubernetes# cat >> audit-policy.yaml <<EOF
> # Log all requests at the Metadata level.
> apiVersion: audit.k8s.io/v1beta1
> kind: Policy
> rules:
> - level: Metadata
> EOF
root@Master:/etc/kubernetes# scp audit-policy.yaml zdyfjh@192.168.10.11:/etc/kubernetes/ssl/  
root@Master:/etc/kubernetes# scp audit-policy.yaml zdyfjh@192.168.10.12:/etc/kubernetes/ssl/    
root@Master:/etc/kubernetes# scp audit-policy.yaml zdyfjh@192.168.10.13:/etc/kubernetes/ssl/   
root@Master:/etc/kubernetes# scp audit-policy.yaml zdyfjh@192.168.10.14:/etc/kubernetes/ssl/  
root@Master:/etc/kubernetes# scp audit-policy.yaml zdyfjh@192.168.10.15:/etc/kubernetes/ssl/
```

* ### 配置APIServer的service文件并启动APIServer

```shell
root@Master:/etc/kubernetes# vim /etc/systemd/system/kube-apiserver.service
root@Master:/etc/kubernetes# cat /etc/systemd/system/kube-apiserver.service
[Unit]
Description=Kubernetes API Server
Documentation=https://github.com/GoogleCloudPlatform/kubernetes
After=network.target

[Service]
User=root
ExecStart=/usr/local/bin/kube-apiserver \
  --admission-control=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,ResourceQuota,NodeRestriction \
  --advertise-address=192.168.10.10 \
  --allow-privileged=true \
  --apiserver-count=1 \
  --audit-policy-file=/etc/kubernetes/audit-policy.yaml \
  --audit-log-maxage=30 \
  --audit-log-maxbackup=3 \
  --audit-log-maxsize=100 \
  --audit-log-path=/var/log/kubernetes/audit.log \
  --authorization-mode=Node,RBAC \
  --bind-address=0.0.0.0 \
  --secure-port=6443 \
  --client-ca-file=/etc/kubernetes/ssl/ca.pem \
  --enable-swagger-ui=true \
  --etcd-cafile=/etc/kubernetes/ssl/ca.pem \
  --etcd-certfile=/etc/kubernetes/ssl/etcd.pem \
  --etcd-keyfile=/etc/kubernetes/ssl/etcd-key.pem \
  --etcd-servers=https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379 \
  --event-ttl=1h \
  --kubelet-https=true \
  --insecure-bind-address=192.168.10.10 \
  --insecure-port=8080 \
  --service-account-key-file=/etc/kubernetes/ssl/ca-key.pem \
  --service-cluster-ip-range=10.254.0.0/18 \
  --service-node-port-range=30000-32000 \
  --tls-cert-file=/etc/kubernetes/ssl/kubernetes.pem \
  --tls-private-key-file=/etc/kubernetes/ssl/kubernetes-key.pem \
  --enable-bootstrap-token-auth \
  --token-auth-file=/etc/kubernetes/token.csv \
  --v=1
Restart=on-failure
RestartSec=5
Type=notify
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
root@Master:/etc/kubernetes# systemctl daemon-reload
root@Master:/etc/kubernetes# systemctl enable kube-apiserver
Created symlink from /etc/systemd/system/multi-user.target.wants/kube-apiserver.service to /etc/systemd/system/kube-apiserver.service.
root@Master:/etc/kubernetes# systemctl start kube-apiserver
root@Master:/etc/kubernetes# systemctl status kube-apiserver
● kube-apiserver.service - Kubernetes API Server
   Loaded: loaded (/etc/systemd/system/kube-apiserver.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 15:49:49 CST; 8s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 38924 (kube-apiserver)
    Tasks: 55
   Memory: 227.7M
      CPU: 8.712s
   CGroup: /system.slice/kube-apiserver.service
           └─38924 /usr/local/bin/kube-apiserver --admission-control=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,Resour

Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.011284   38924 storage_rbac.go:267] created role.rbac.authorization.k8s.io/syste
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.047667   38924 storage_rbac.go:267] created role.rbac.authorization.k8s.io/syste
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.091192   38924 storage_rbac.go:267] created role.rbac.authorization.k8s.io/syste
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.125045   38924 controller.go:538] quota admission added evaluator for: {rbac.aut
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.128525   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.167760   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.208015   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.247990   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.288011   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
Oct 23 15:49:52 Master kube-apiserver[38924]: I1023 15:49:52.327823   38924 storage_rbac.go:297] created rolebinding.rbac.authorization.k8s.i
```

* ### 配置kube-controller的service文件并启动

```shell
root@Master:/etc/kubernetes# vim /etc/systemd/system/kube-controller-manager.service
root@Master:/etc/kubernetes# cat /etc/systemd/system/kube-controller-manager.service 
[Unit]
Description=Kubernetes Controller Manager
Documentation=https://github.com/GoogleCloudPlatform/kubernetes

[Service]
ExecStart=/usr/local/bin/kube-controller-manager \
  --address=0.0.0.0 \
  --master=http://192.168.10.10:8080 \
  --allocate-node-cidrs=true \
  --service-cluster-ip-range=10.254.0.0/18 \
  --cluster-cidr=10.254.64.0/18 \
  --cluster-name=kubernetes \
  --service-account-private-key-file=/etc/kubernetes/ssl/ca-key.pem \
  --root-ca-file=/etc/kubernetes/ssl/ca.pem \
  --cluster-signing-cert-file=/etc/kubernetes/ssl/ca.pem \
  --cluster-signing-key-file=/etc/kubernetes/ssl/ca-key.pem \
  --leader-elect=true \
  --v=1
Restart=on-failure
RestartSec=5

[Install]
WantedBy=multi-user.target
root@Master:/etc/kubernetes# systemctl daemon-reload
root@Master:/etc/kubernetes# systemctl enable kube-controller-manager
Created symlink from /etc/systemd/system/multi-user.target.wants/kube-controller-manager.service to /etc/systemd/system/kube-controller-manager.service.
root@Master:/etc/kubernetes# systemctl start kube-controller-manager
root@Master:/etc/kubernetes# systemctl status kube-controller-manager
● kube-controller-manager.service - Kubernetes Controller Manager
   Loaded: loaded (/etc/systemd/system/kube-controller-manager.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 15:53:31 CST; 6s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 39196 (kube-controller)
    Tasks: 30
   Memory: 12.3M
      CPU: 180ms
   CGroup: /system.slice/kube-controller-manager.service
           └─39196 /usr/local/bin/kube-controller-manager --address=0.0.0.0 --master=http://192.168.10.10:8080 --allocate-node-cidrs=true --s

Oct 23 15:53:31 Master systemd[1]: Started Kubernetes Controller Manager.
Oct 23 15:53:31 Master kube-controller-manager[39196]: I1023 15:53:31.803429   39196 controllermanager.go:108] Version: v1.9.6
Oct 23 15:53:31 Master kube-controller-manager[39196]: I1023 15:53:31.804194   39196 leaderelection.go:174] attempting to acquire leader leas
Oct 23 15:53:31 Master kube-controller-manager[39196]: E1023 15:53:31.805463   39196 leaderelection.go:224] error retrieving resource lock ku
Oct 23 15:53:35 Master kube-controller-manager[39196]: E1023 15:53:35.257755   39196 leaderelection.go:224] error retrieving resource lock ku
```

* ### 配置scheduler的service文件并启动

```shell
root@Master:/etc/kubernetes# vim /etc/systemd/system/kube-scheduler.service
root@Master:/etc/kubernetes# cat /etc/systemd/system/kube-scheduler.service 
[Unit]
Description=Kubernetes Scheduler
Documentation=https://github.com/GoogleCloudPlatform/kubernetes

[Service]
ExecStart=/usr/local/bin/kube-scheduler \
  --address=0.0.0.0 \
  --master=http://192.168.10.10:8080 \
  --leader-elect=true \
  --v=1
Restart=on-failure
RestartSec=5

[Install]
WantedBy=multi-user.target
root@Master:/etc/kubernetes# systemctl daemon-reload
root@Master:/etc/kubernetes# systemctl enable kube-scheduler
Created symlink from /etc/systemd/system/multi-user.target.wants/kube-scheduler.service to /etc/systemd/system/kube-scheduler.service.
root@Master:/etc/kubernetes# systemctl start kube-scheduler
root@Master:/etc/kubernetes# systemctl status kube-scheduler
● kube-scheduler.service - Kubernetes Scheduler
   Loaded: loaded (/etc/systemd/system/kube-scheduler.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 15:57:13 CST; 6s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 39435 (kube-scheduler)
    Tasks: 33
   Memory: 14.5M
      CPU: 253ms
   CGroup: /system.slice/kube-scheduler.service
           └─39435 /usr/local/bin/kube-scheduler --address=0.0.0.0 --master=http://192.168.10.10:8080 --leader-elect=true --v=1

Oct 23 15:57:18 Master kube-scheduler[39435]: E1023 15:57:18.155660   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.146464   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.147634   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.148800   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.149888   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.151873   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.152995   39435 reflector.go:205] k8s.io/kubernetes/plugin/cmd/kube-scheduler/app
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.154028   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.155108   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
Oct 23 15:57:19 Master kube-scheduler[39435]: E1023 15:57:19.156241   39435 reflector.go:205] k8s.io/kubernetes/vendor/k8s.io/client-go/infor
```

* ### 验证安装工作

```shell
root@Master:/etc/kubernetes# kubectl get componentstatuses
NAME                 STATUS    MESSAGE             ERROR
scheduler            Healthy   ok                  
controller-manager   Healthy   ok                  
etcd-2               Healthy   {"health":"true"}   
etcd-1               Healthy   {"health":"true"}   
etcd-5               Healthy   {"health":"true"}   
etcd-0               Healthy   {"health":"true"}   
etcd-4               Healthy   {"health":"true"}   
etcd-3               Healthy   {"health":"true"}   
```

* ### 配置kubelet

```shell
root@Master:/etc/kubernetes# kubectl create clusterrolebinding kubelet-bootstrap --clusterrole=system:node-bootstrapper --user=kubelet-bootstrap
clusterrolebinding "kubelet-bootstrap" created
root@Master:/etc/kubernetes# kubectl config set-cluster kubernetes \
>   --certificate-authority=/etc/kubernetes/ssl/ca.pem \
>   --embed-certs=true \
>   --server=https://192.168.10.10:6443 \
>   --kubeconfig=bootstrap.kubeconfig
Cluster "kubernetes" set.
root@Master:/etc/kubernetes# kubectl config set-credentials kubelet-bootstrap \
>   --token=f27f606aa0bb8bea34b1e22aabd3aab8 \
>   --kubeconfig=bootstrap.kubeconfig
User "kubelet-bootstrap" set.
root@Master:/etc/kubernetes# kubectl config set-context default \
>   --cluster=kubernetes \
>   --user=kubelet-bootstrap \
>   --kubeconfig=bootstrap.kubeconfig
Context "default" created.
root@Master:/etc/kubernetes# kubectl config use-context default --kubeconfig=bootstrap.kubeconfig
Switched to context "default".
root@Master:/etc/kubernetes# ls
audit-policy.yaml  bootstrap.kubeconfig  ssl  token.csv
root@Master:/etc/kubernetes# mv bootstrap.kubeconfig /etc/kubernetes/
mv: 'bootstrap.kubeconfig' and '/etc/kubernetes/bootstrap.kubeconfig' are the same file
root@Master:/etc/kubernetes# scp bootstrap.kubeconfig zdyfjh@192.168.10.11:/etc/kubernetes/ssl/  
root@Master:/etc/kubernetes# scp bootstrap.kubeconfig zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/etc/kubernetes# scp bootstrap.kubeconfig zdyfjh@192.168.10.13:/etc/kubernetes/ssl/  
root@Master:/etc/kubernetes# scp bootstrap.kubeconfig zdyfjh@192.168.10.14:/etc/kubernetes/ssl/   
root@Master:/etc/kubernetes# scp bootstrap.kubeconfig zdyfjh@192.168.10.15:/etc/kubernetes/ssl/ 
```

* ### 创建kubelet的service文件并启动

```shell
root@Master:/etc/kubernetes# mkdir /var/lib/kubelet
root@Master:/etc/kubernetes# vim /etc/systemd/system/kubelet.service
root@Master:/etc/kubernetes# cat /etc/systemd/system/kubelet.service 
[Unit]
Description=Kubernetes Kubelet
Documentation=https://github.com/GoogleCloudPlatform/kubernetes
After=docker.service
Requires=docker.service

[Service]
WorkingDirectory=/var/lib/kubelet
ExecStart=/usr/local/bin/kubelet \
  --cgroup-driver=cgroupfs \
  --hostname-override=Master \ #改成自己的hostname
  --pod-infra-container-image=jicki/pause-amd64:3.0 \ #需要预先从仓库里把这个镜像pull到本地
  --experimental-bootstrap-kubeconfig=/etc/kubernetes/bootstrap.kubeconfig \
  --kubeconfig=/etc/kubernetes/kubelet.kubeconfig \
  --cert-dir=/etc/kubernetes/ssl \
  --cluster_dns=10.254.0.2 \
  --cluster_domain=cluster.local \
  --hairpin-mode=promiscuous-bridge \
  --allow-privileged=true \
  --fail-swap-on=false \
  --serialize-image-pulls=false \
  --logtostderr=true \
  --max-pods=512 \
  --v=1

[Install]
WantedBy=multi-user.target
root@Master:/etc/kubernetes# docker pull jicki/pause-amd64:3.0
3.0: Pulling from jicki/pause-amd64
a3ed95caeb02: Pull complete 
f11233434377: Pull complete 
Digest: sha256:3b3a29e3c90ae7762bdf587d19302e62485b6bef46e114b741f7d75dba023bd3
Status: Downloaded newer image for jicki/pause-amd64:3.0
root@Master:/etc/kubernetes# systemctl daemon-reload
root@Master:/etc/kubernetes# systemctl enable kubelet
Created symlink from /etc/systemd/system/multi-user.target.wants/kubelet.service to /etc/systemd/system/kubelet.service.
root@Master:/etc/kubernetes# systemctl start kubelet
root@Master:/etc/kubernetes# systemctl status kubelet
● kubelet.service - Kubernetes Kubelet
   Loaded: loaded (/etc/systemd/system/kubelet.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 16:19:40 CST; 8s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 40677 (kubelet)
    Tasks: 30
   Memory: 16.8M
      CPU: 620ms
   CGroup: /system.slice/kubelet.service
           └─40677 /usr/local/bin/kubelet --cgroup-driver=cgroupfs --hostname-override=Master --pod-infra-container-image=jicki/pause-amd64:3

Oct 23 16:19:40 Master systemd[1]: Started Kubernetes Kubelet.
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.198672   40677 feature_gate.go:226] feature gates: &{{} map[]}
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.198855   40677 controller.go:114] kubelet config controller: starting controller
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.198872   40677 controller.go:118] kubelet config controller: validating combination of 
Oct 23 16:19:40 Master kubelet[40677]: W1023 16:19:40.615114   40677 cni.go:171] Unable to update cni config: No networks found in /etc/cni/n
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.620213   40677 server.go:182] Version: v1.9.6
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.620268   40677 feature_gate.go:226] feature gates: &{{} map[]}
Oct 23 16:19:40 Master kubelet[40677]: I1023 16:19:40.620378   40677 plugins.go:101] No cloud provider specified.
```

* ### 配置TLS认证

```shell
root@Master:/etc/kubernetes/ssl# kubectl get csr
NAME                                                   AGE       REQUESTOR           CONDITION
node-csr-eufLGPzV7r2HVCmqCAanrltdfrcf1c96ixfn0gKACL0   1m        kubelet-bootstrap   Pending
root@Master:/etc/kubernetes/ssl# kubectl get csr | grep Pending | awk '{print $1}' | xargs kubectl certificate approve
certificatesigningrequest "node-csr-eufLGPzV7r2HVCmqCAanrltdfrcf1c96ixfn0gKACL0" approved
root@Master:/etc/kubernetes/ssl# kubectl get csr
NAME                                                   AGE       REQUESTOR           CONDITION
node-csr-eufLGPzV7r2HVCmqCAanrltdfrcf1c96ixfn0gKACL0   1m        kubelet-bootstrap   Approved,Issued
root@Master:/etc/kubernetes/ssl# kubectl get node
NAME      STATUS    ROLES     AGE       VERSION
master    Ready     <none>    1h        v1.9.6
root@Master:/etc/kubernetes# ls /etc/kubernetes/kubelet.kubeconfig 
/etc/kubernetes/kubelet.kubeconfig
root@Master:/etc/kubernetes# ls /etc/kubernetes/ssl/kubelet*
/etc/kubernetes/ssl/kubelet-client.crt  /etc/kubernetes/ssl/kubelet.crt
/etc/kubernetes/ssl/kubelet-client.key  /etc/kubernetes/ssl/kubelet.key
```

* ### 配置kube-proxy证书

```shell
root@Master:/etc/kubernetes# cd /opt/ssl
root@Master:/opt/ssl# vim kube-proxy-csr.json
root@Master:/opt/ssl# cat kube-proxy-csr.json 
{
  "CN": "system:kube-proxy",
  "hosts": [],
  "key": {
    "algo": "rsa",
    "size": 2048
  },
  "names": [
    {
      "C": "CN",
      "ST": "Beijing",
      "L": "Beijing",
      "O": "k8s",
      "OU": "System"
    }
  ]
}
root@Master:/opt/ssl# /opt/local/cfssl/cfssl gencert -ca=/etc/kubernetes/ssl/ca.pem \
>   -ca-key=/etc/kubernetes/ssl/ca-key.pem \
>   -config=/opt/ssl/config.json \
>   -profile=kubernetes  kube-proxy-csr.json | /opt/local/cfssl/cfssljson -bare kube-proxy
2018/10/23 20:06:17 [INFO] generate received request
2018/10/23 20:06:17 [INFO] received CSR
2018/10/23 20:06:17 [INFO] generating key: rsa-2048
2018/10/23 20:06:17 [INFO] encoded CSR
2018/10/23 20:06:17 [INFO] signed certificate with serial number 36413578466703637328292670519988450625987607631
2018/10/23 20:06:17 [WARNING] This certificate lacks a "hosts" field. This makes it unsuitable for
websites. For more information see the Baseline Requirements for the Issuance and Management
of Publicly-Trusted Certificates, v.1.1.6, from the CA/Browser Forum (https://cabforum.org);
specifically, section 10.2.3 ("Information Requirements").
root@Master:/opt/ssl# ls kube-proxy*
kube-proxy.csr  kube-proxy-csr.json  kube-proxy-key.pem  kube-proxy.pem
root@Master:/opt/ssl# cp kube-proxy* /etc/kubernetes/ssl/
root@Master:/opt/ssl# scp kube-proxy* zdyfjh@192.168.10.11:/etc/kubernetes/ssl/ 
root@Master:/opt/ssl# scp kube-proxy* zdyfjh@192.168.10.12:/etc/kubernetes/ssl/   
root@Master:/opt/ssl# scp kube-proxy* zdyfjh@192.168.10.13:/etc/kubernetes/ssl/  
root@Master:/opt/ssl# scp kube-proxy* zdyfjh@192.168.10.14:/etc/kubernetes/ssl/    
root@Master:/opt/ssl# scp kube-proxy* zdyfjh@192.168.10.15:/etc/kubernetes/ssl/
root@Master:/opt/ssl# kubectl config set-cluster kubernetes \
>   --certificate-authority=/etc/kubernetes/ssl/ca.pem \
>   --embed-certs=true \
>   --server=https://192.168.10.10:6443 \
>   --kubeconfig=kube-proxy.kubeconfig
Cluster "kubernetes" set.
root@Master:/opt/ssl# kubectl config set-credentials kube-proxy \
>   --client-certificate=/etc/kubernetes/ssl/kube-proxy.pem \
>   --client-key=/etc/kubernetes/ssl/kube-proxy-key.pem \
>   --embed-certs=true \
>   --kubeconfig=kube-proxy.kubeconfig
User "kube-proxy" set.
root@Master:/opt/ssl# kubectl config set-context default \
>   --cluster=kubernetes \
>   --user=kube-proxy \
>   --kubeconfig=kube-proxy.kubeconfig
Context "default" created.
root@Master:/opt/ssl# kubectl config use-context default --kubeconfig=kube-proxy.kubeconfig
Switched to context "default".
root@Master:/opt/ssl# cp kube-proxy.kubeconfig /etc/kubernetes/
root@Master:/opt/ssl# cd /etc/kubernetes/
root@Master:/etc/kubernetes# ls
audit-policy.yaml  bootstrap.kubeconfig  kubelet.kubeconfig  kube-proxy.kubeconfig  ssl  token.csv
root@Master:/etc/kubernetes# scp kube-proxy.kubeconfig zdyfjh@192.168.10.11:/etc/kubernetes/ssl    
root@Master:/etc/kubernetes# scp kube-proxy.kubeconfig zdyfjh@192.168.10.12:/etc/kubernetes/ssl   
root@Master:/etc/kubernetes# scp kube-proxy.kubeconfig zdyfjh@192.168.10.13:/etc/kubernetes/ssl   
root@Master:/etc/kubernetes# scp kube-proxy.kubeconfig zdyfjh@192.168.10.14:/etc/kubernetes/ssl   
root@Master:/etc/kubernetes# scp kube-proxy.kubeconfig zdyfjh@192.168.10.15:/etc/kubernetes/ssl                   
```

* ### 配置kube-proxy的service文件并启动

```shell
root@Master:/etc/kubernetes# mkdir -p /var/lib/kube-proxy
root@Master:/etc/kubernetes# vim /etc/systemd/system/kube-proxy.service
root@Master:/etc/kubernetes# cat /etc/systemd/system/kube-proxy.service 
[Unit]
Description=Kubernetes Kube-Proxy Server
Documentation=https://github.com/GoogleCloudPlatform/kubernetes
After=network.target

[Service]
WorkingDirectory=/var/lib/kube-proxy
ExecStart=/usr/local/bin/kube-proxy \
  --bind-address=192.168.10.10 \
  --hostname-override=k8s-master-47 \
  --cluster-cidr=10.254.64.0/18 \
  --masquerade-all \
  --feature-gates=SupportIPVSProxyMode=true \
  --proxy-mode=ipvs \
  --ipvs-min-sync-period=5s \
  --ipvs-sync-period=5s \
  --ipvs-scheduler=rr \
  --kubeconfig=/etc/kubernetes/kube-proxy.kubeconfig \
  --logtostderr=true \
  --v=1
Restart=on-failure
RestartSec=5
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
root@Master:/etc/kubernetes# systemctl daemon-reload
root@Master:/etc/kubernetes# systemctl enable kube-proxy
Created symlink from /etc/systemd/system/multi-user.target.wants/kube-proxy.service to /etc/systemd/system/kube-proxy.service.
root@Master:/etc/kubernetes# systemctl start kube-proxy
root@Master:/etc/kubernetes# systemctl status kube-proxy
● kube-proxy.service - Kubernetes Kube-Proxy Server
   Loaded: loaded (/etc/systemd/system/kube-proxy.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 20:21:56 CST; 37s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 13444 (kube-proxy)
    Tasks: 0
   Memory: 13.6M
      CPU: 261ms
   CGroup: /system.slice/kube-proxy.service
           ‣ 13444 /usr/local/bin/kube-proxy --bind-address=192.168.10.10 --hostname-override=k8s-master-47 --cluster-cidr=10.254.64.0/18 --m

Oct 23 20:21:56 Master kube-proxy[13444]: I1023 20:21:56.990793   13444 conntrack.go:83] Setting conntrack hashsize to 393216
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.045973   13444 conntrack.go:98] Set sysctl 'net/netfilter/nf_conntrack_tcp_timeout_e
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.046196   13444 conntrack.go:98] Set sysctl 'net/netfilter/nf_conntrack_tcp_timeout_c
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.046457   13444 config.go:102] Starting endpoints config controller
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.046495   13444 controller_utils.go:1019] Waiting for caches to sync for endpoints co
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.046592   13444 config.go:202] Starting service config controller
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.046674   13444 controller_utils.go:1019] Waiting for caches to sync for service conf
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.146696   13444 controller_utils.go:1026] Caches are synced for endpoints config cont
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.146855   13444 controller_utils.go:1026] Caches are synced for service config contro
Oct 23 20:21:57 Master kube-proxy[13444]: I1023 20:21:57.146960   13444 proxier.go:329] Adding new service port "default/kubernetes:https" at
root@Master:/etc/kubernetes# ipvsadm -L -n
IP Virtual Server version 1.2.1 (size=4096)
Prot LocalAddress:Port Scheduler Flags
  -> RemoteAddress:Port           Forward Weight ActiveConn InActConn
root@Master:/etc/kubernetes# cd /etc/kubernetes/ssl/
root@Master:/etc/kubernetes/ssl# scp ca.pem kube-proxy.pem kube-proxy-key.pem zdyfjh@192.168.10.11:/etc/kubernetes/ssl 
root@Master:/etc/kubernetes/ssl# scp ca.pem kube-proxy.pem kube-proxy-key.pem zdyfjh@192.168.10.12:/etc/kubernetes/ssl 
root@Master:/etc/kubernetes/ssl# scp ca.pem kube-proxy.pem kube-proxy-key.pem zdyfjh@192.168.10.13:/etc/kubernetes/ssl
root@Master:/etc/kubernetes/ssl# scp ca.pem kube-proxy.pem kube-proxy-key.pem zdyfjh@192.168.10.14:/etc/kubernetes/ssl
root@Master:/etc/kubernetes/ssl# scp ca.pem kube-proxy.pem kube-proxy-key.pem zdyfjh@192.168.10.15:/etc/kubernetes/ssl
```

## Slave节点上

* ### 安装Kubernetes相关组件及配置

```shell
root@Slave:/home/zdyfjh/download# tar -zxvf kubernetes-server-linux-amd64.tar.gz 
root@Slave:/home/zdyfjh/download# ls
etcd-v3.3.10-linux-amd64         kubernetes
etcd-v3.3.10-linux-amd64.tar.gz  kubernetes-server-linux-amd64.tar.gz
root@Slave:/home/zdyfjh/download# cd ./kubernetes
root@Slave:/home/zdyfjh/download/kubernetes# ls
LICENSES  addons  kubernetes-src.tar.gz  server
root@Slave:/home/zdyfjh/download/kubernetes# cp server/bin/{kube-proxy,kubelet,kubectl} /usr/local/bin/
root@Slave:/home/zdyfjh/download/kubernetes# cd ~
root@Slave:~# mkdir -p .kube
root@Slave:~# mv /home/zdyfjh/download/config .kube
root@Slave:~# cd .kube
root@Slave:~/.kube# chown -R root:root config 
root@Slave:~/.kube# ll
total 16
drwxr-xr-x 2 root root 4096 Oct 23 15:09 ./
drwx------ 6 root root 4096 Oct 23 15:09 ../
-rw------- 1 root root 6287 Oct 23 15:09 config
```

* ### 移动Token文件及高级审核文件

```shell
root@Slave:/home/zdyfjh# cd /etc/kubernetes/ssl/
root@Slave:/etc/kubernetes/ssl# ls
admin-key.pem      ca-key.pem  config.json   etcd.pem            token.csv
admin.pem          ca.csr      csr.json      kubernetes-key.pem
audit-policy.yaml  ca.pem      etcd-key.pem  kubernetes.pem
root@Slave:/etc/kubernetes/ssl# mv token.csv ../
root@Slave:/etc/kubernetes/ssl# mv audit-policy.yaml ../
root@Slave:/etc/kubernetes/ssl# cd ../
root@Slave:/etc/kubernetes# chown -R root:root audit-policy.yaml 
root@Slave:/etc/kubernetes# chown -R root:root token.csv 
root@Slave:/etc/kubernetes# ll
total 28
drwxr-xr-x   3 root   root    4096 Oct 23 15:35 ./
drwxr-xr-x 138 root   root   12288 Oct 23 10:27 ../
-rw-r--r--   1 root   root     113 Oct 23 15:33 audit-policy.yaml
drwxr-xr-x   2 zdyfjh zdyfjh  4096 Oct 23 15:35 ssl/
-rw-r--r--   1 root   root      84 Oct 23 15:28 token.csv
```

* ### 移动配置文件并创建工作目录

```shell
root@Slave:/home/zdyfjh# cd /etc/kubernetes/ssl/
root@Slave:/etc/kubernetes/ssl# mv bootstrap.kubeconfig ../
root@Slave:/etc/kubernetes/ssl# cd ../                
root@Slave:/etc/kubernetes# chown -R root:root bootstrap.kubeconfig 
root@Slave:/etc/kubernetes# ll
total 32
drwxr-xr-x   3 root   root    4096 Oct 23 16:10 ./
drwxr-xr-x 138 root   root   12288 Oct 23 10:27 ../
-rw-r--r--   1 root   root     113 Oct 23 15:33 audit-policy.yaml
-rw-------   1 root   root    2189 Oct 23 16:05 bootstrap.kubeconfig
drwxr-xr-x   2 zdyfjh zdyfjh  4096 Oct 23 16:10 ssl/
-rw-r--r--   1 root   root      84 Oct 23 15:28 token.csv
```

* ### 移动kube-proxy配置文件

```shell
root@Slave:/home/zdyfjh# cd /etc/kubernetes/ssl
root@Slave:/etc/kubernetes/ssl# mv kube-proxy.kubeconfig ../
root@Slave:/etc/kubernetes/ssl# cd ../
root@Slave:/etc/kubernetes# chown -R root:root kube-proxy.kubeconfig 
root@Slave:/etc/kubernetes# ll
total 40
drwxr-xr-x   3 root   root    4096 Oct 23 20:15 ./
drwxr-xr-x 135 root   root   12288 Oct 23 10:30 ../
-rw-r--r--   1 root   root     113 Oct 23 15:34 audit-policy.yaml
-rw-------   1 root   root    2185 Oct 23 19:41 bootstrap.kubeconfig
-rw-------   1 root   root    6295 Oct 23 20:12 kube-proxy.kubeconfig
drwxr-xr-x   2 zdyfjh zdyfjh  4096 Oct 23 20:15 ssl/
-rw-r--r--   1 root   root      84 Oct 23 19:33 token.csv
```

* ### 创建kubelet的service文件并启动

```shell
root@Slave:/home/zdyfjh# cd /etc/kubernetes/
root@Slave:/etc/kubernetes# ls
audit-policy.yaml  bootstrap.kubeconfig  kube-proxy.kubeconfig  ssl  token.csv
root@Slave:/etc/kubernetes# mkdir /var/lib/kubelet
root@Slave:/etc/kubernetes# vim /etc/systemd/system/kubelet.service
root@Slave:/etc/kubernetes# cat /etc/systemd/system/kubelet.service 
[Unit]
Description=Kubernetes Kubelet
Documentation=https://github.com/GoogleCloudPlatform/kubernetes
After=docker.service
Requires=docker.service

[Service]
WorkingDirectory=/var/lib/kubelet
ExecStart=/usr/local/bin/kubelet \
  --cgroup-driver=cgroupfs \
  --hostname-override=Slave1 \ #这里改为主机名
  --pod-infra-container-image=jicki/pause-amd64:3.0 \ #最好提前pull该镜像到本地
  --experimental-bootstrap-kubeconfig=/etc/kubernetes/bootstrap.kubeconfig \
  --kubeconfig=/etc/kubernetes/kubelet.kubeconfig \
  --cert-dir=/etc/kubernetes/ssl \
  --cluster_dns=10.254.0.2 \
  --cluster_domain=cluster.local \
  --hairpin-mode promiscuous-bridge \
  --allow-privileged=true \
  --fail-swap-on=false \
  --serialize-image-pulls=false \
  --logtostderr=true \
  --max-pods=512 \
  --v=1

[Install]
WantedBy=multi-user.target
root@Slave1:/etc/kubernetes# docker pull jicki/pause-amd64:3.0
3.0: Pulling from jicki/pause-amd64
a3ed95caeb02: Pull complete 
f11233434377: Pull complete 
Digest: sha256:3b3a29e3c90ae7762bdf587d19302e62485b6bef46e114b741f7d75dba023bd3
Status: Downloaded newer image for jicki/pause-amd64:3.0
root@Slave1:/etc/kubernetes/ssl# systemctl daemon-reload
root@Slave1:/etc/kubernetes/ssl# systemctl enable kubelet
root@Slave1:/etc/kubernetes/ssl# systemctl start kubelet
root@Slave1:/etc/kubernetes/ssl# systemctl status kubelet
● kubelet.service - Kubernetes Kubelet
   Loaded: loaded (/etc/systemd/system/kubelet.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 20:56:04 CST; 3min 11s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 12262 (kubelet)
   CGroup: /system.slice/kubelet.service
           └─12262 /usr/local/bin/kubelet --cgroup-driver=cgroupfs --hostname-override=Slave1 --pod-infra-container-image=jicki/pause-amd64:3

Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.878631   12262 feature_gate.go:226] feature gates: &{{} map[]}
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.878805   12262 controller.go:114] kubelet config controller: starting controller
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.878817   12262 controller.go:118] kubelet config controller: validating combination of 
Oct 23 20:56:04 Slave1 kubelet[12262]: W1023 20:56:04.885009   12262 cni.go:171] Unable to update cni config: No networks found in /etc/cni/n
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.889856   12262 server.go:182] Version: v1.9.6
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.889931   12262 feature_gate.go:226] feature gates: &{{} map[]}
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.890039   12262 plugins.go:101] No cloud provider specified.
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.941488   12262 csr.go:105] csr for this node already exists, reusing
Oct 23 20:56:04 Slave1 kubelet[12262]: I1023 20:56:04.948060   12262 csr.go:113] csr for this node is still valid
Oct 23 20:59:06 Slave1 systemd[1]: Started Kubernetes Kubelet.
```

## Master节点上的操做（注意：每个从节点添加后都需要在主节点上进行该操作）

* ### 信任证书

```shell
root@Master:/etc/kubernetes# kubectl get csr
NAME                                                   AGE       REQUESTOR           CONDITION
node-csr-DBQb9r6wLENeG7oTjRMzMhsvf34FBkG92tMg9BhPhpA   17m       kubelet-bootstrap   Pending
node-csr-eufLGPzV7r2HVCmqCAanrltdfrcf1c96ixfn0gKACL0   1h        kubelet-bootstrap   Approved,Issued
root@Master:/etc/kubernetes# kubectl get csr | grep Pending | awk '{print $1}' | xargs kubectl certificate approve
certificatesigningrequest "node-csr-DBQb9r6wLENeG7oTjRMzMhsvf34FBkG92tMg9BhPhpA" approved
root@Master:/etc/kubernetes# kubectl get csr
NAME                                                   AGE       REQUESTOR           CONDITION
node-csr-DBQb9r6wLENeG7oTjRMzMhsvf34FBkG92tMg9BhPhpA   20m       kubelet-bootstrap   Approved,Issued
node-csr-eufLGPzV7r2HVCmqCAanrltdfrcf1c96ixfn0gKACL0   1h        kubelet-bootstrap   Approved,Issued
```

## Slave节点上的操作

* ### 配置kube-proxy的service文件并启动

```shell
root@Slave1:/etc/kubernetes# mkdir -p /var/lib/kube-proxy
root@Slave1:/etc/kubernetes# vim /etc/systemd/system/kube-proxy.service
root@Slave1:/etc/kubernetes# cat /etc/systemd/system/kube-proxy.service
[Unit]
Description=Kubernetes Kube-Proxy Server
Documentation=https://github.com/GoogleCloudPlatform/kubernetes
After=network.target

[Service]
WorkingDirectory=/var/lib/kube-proxy
ExecStart=/usr/local/bin/kube-proxy \
  --bind-address=192.168.10.11 \ #需要改成自己的IP
  --hostname-override=Slave1 \ #改成自己的主机名
  --cluster-cidr=10.254.64.0/18 \
  --masquerade-all \
  --feature-gates=SupportIPVSProxyMode=true \
  --proxy-mode=ipvs \
  --ipvs-min-sync-period=5s \
  --ipvs-sync-period=5s \
  --ipvs-scheduler=rr \
  --kubeconfig=/etc/kubernetes/kube-proxy.kubeconfig \
  --logtostderr=true \
  --v=2
Restart=on-failure
RestartSec=5
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
root@Slave1:/etc/kubernetes# systemctl daemon-reload
root@Slave1:/etc/kubernetes# systemctl start kube-proxy
root@Slave1:/etc/kubernetes# systemctl status kube-proxy
● kube-proxy.service - Kubernetes Kube-Proxy Server
   Loaded: loaded (/etc/systemd/system/kube-proxy.service; disabled; vendor preset: enabled)
   Active: active (running) since Tue 2018-10-23 21:08:06 CST; 7s ago
     Docs: https://github.com/GoogleCloudPlatform/kubernetes
 Main PID: 13300 (kube-proxy)
    Tasks: 0
   Memory: 13.9M
      CPU: 254ms
   CGroup: /system.slice/kube-proxy.service
           ‣ 13300 /usr/local/bin/kube-proxy --bind-address=192.168.10.11 --hostname-override=k8s-node-1 --cluster-cidr=10.254.64.0/18 --masq

Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.345881   13300 conntrack.go:98] Set sysctl 'net/netfilter/nf_conntrack_tcp_timeout_e
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.345995   13300 conntrack.go:98] Set sysctl 'net/netfilter/nf_conntrack_tcp_timeout_c
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.346180   13300 config.go:102] Starting endpoints config controller
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.346219   13300 controller_utils.go:1019] Waiting for caches to sync for endpoints co
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.346237   13300 config.go:202] Starting service config controller
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.346371   13300 controller_utils.go:1019] Waiting for caches to sync for service conf
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.446382   13300 controller_utils.go:1026] Caches are synced for endpoints config cont
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.446490   13300 proxier.go:984] Not syncing iptables until Services and Endpoints hav
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.446596   13300 controller_utils.go:1026] Caches are synced for service config contro
Oct 23 21:08:06 Slave1 kube-proxy[13300]: I1023 21:08:06.446689   13300 proxier.go:329] Adding new service port "default/kubernetes:https" at
```

# 安装Flannel网络插件

## Master节点上的操作

* ### 下载并安装Flannel

```shell
root@Master:/home/zdyfjh/download# wget https://github.com/coreos/flannel/releases/download/v0.9.0/flannel-v0.9.0-linux-amd64.tar.gz
root@Master:/home/zdyfjh/download# ls
etcd-v3.3.10-linux-amd64         flannel-v0.9.0-linux-amd64.tar.gz  kubernetes-server-linux-amd64.tar.gz
etcd-v3.3.10-linux-amd64.tar.gz  kubernetes
root@Master:/home/zdyfjh/download# tar -zxvf flannel-v0.9.0-linux-amd64.tar.gz 
flanneld
mk-docker-opts.sh
README.md
root@Master:/home/zdyfjh/download# ls
etcd-v3.3.10-linux-amd64         flanneld                           kubernetes                            mk-docker-opts.sh
etcd-v3.3.10-linux-amd64.tar.gz  flannel-v0.9.0-linux-amd64.tar.gz  kubernetes-server-linux-amd64.tar.gz  README.md
root@Master:/home/zdyfjh/download# mv flanneld /usr/local/bin/
root@Master:/home/zdyfjh/download# mv mk-docker-opts.sh /usr/local/bin/
root@Master:/home/zdyfjh/download# scp flannel-v0.9.0-linux-amd64.tar.gz zdyfjh@192.168.10.11:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp flannel-v0.9.0-linux-amd64.tar.gz zdyfjh@192.168.10.12:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp flannel-v0.9.0-linux-amd64.tar.gz zdyfjh@192.168.10.13:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp flannel-v0.9.0-linux-amd64.tar.gz zdyfjh@192.168.10.14:/home/zdyfjh/download   
root@Master:/home/zdyfjh/download# scp flannel-v0.9.0-linux-amd64.tar.gz zdyfjh@192.168.10.15:/home/zdyfjh/download 
```

* ### 在Etcd中配置flannel信息

```shell
root@Master:/home/zdyfjh/download# etcdctl --endpoints=https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379 \
>         --cert-file=/etc/kubernetes/ssl/etcd.pem \
>         --ca-file=/etc/kubernetes/ssl/ca.pem \
>         --key-file=/etc/kubernetes/ssl/etcd-key.pem \
>         set /flannel/network/config \ '{"Network":"10.254.64.0/18","SubnetLen":24,"Backend":{"Type":"vxlan"}}'
 {"Network":"10.254.64.0/18","SubnetLen":24,"Backend":{"Type":"vxlan"}}

```

* ### 编写flannel的service文件并启动

```shell
root@Master:/home/zdyfjh/download# vim /etc/systemd/system/flanneld.service
root@Master:/home/zdyfjh/download# cat /etc/systemd/system/flanneld.service 
[Unit]
Description=Flanneld
Documentation=https://github.com/coreos/flannel
After=network.target
After=network-online.target
Wants=network-online.target
After=etcd.service
Before=docker.service
[Service]
User=root
ExecStart=/usr/local/bin/flanneld \
        -etcd-endpoints="https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379" \
        -ip-masq=true \
        -etcd-cafile=/etc/kubernetes/ssl/ca.pem \
        -etcd-certfile=/etc/kubernetes/ssl/etcd.pem \
        -etcd-keyfile=/etc/kubernetes/ssl/etcd-key.pem \
	    -iface=eno4 \ #这里需要改成自己的网卡名称或填入本机的IP地址
        -etcd-prefix=/flannel/network
ExecStartPost=/usr/local/bin/mk-docker-opts.sh -d /etc/default/docker -c #注意最后的/etc/default/docker需要将其中的内容拷贝到docker.service中
Restart=on-failure
Type=notify
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
root@Master:/etc/default# cat /etc/default/docker 
DOCKER_OPTS=" --bip=10.254.69.1/24 --ip-masq=false --mtu=1450"
root@Master:/etc/default# vim /lib/systemd/system/docker.service 
root@Master:/etc/default# cat /lib/systemd/system/docker.service 
...
ExecStart=/usr/bin/dockerd -H fd:// --containerd=/run/containerd/containerd.sock --bip=10.254.69.1/24 --ip-masq=false --mtu=1450 #在末尾加上/etc/default/docker中的启动内容
...
root@Master:/home/zdyfjh/download# systemctl daemon-reload
root@Master:/home/zdyfjh/download# systemctl enable flanneld
root@Master:/home/zdyfjh/download# systemctl start flanneld
root@Master:/home/zdyfjh/download# systemctl restart docker
root@Master:/home/zdyfjh/download# systemctl restart kubelet
root@Master:/home/zdyfjh/download# systemctl restart kube-proxy
root@Master:/home/zdyfjh# iptables -P INPUT ACCEPT
root@Master:/home/zdyfjh# iptables -P FORWARD ACCEPT
root@Master:/home/zdyfjh# iptables -F
root@Master:/home/zdyfjh# iptables -L -n
```

## Slave节点上的操作

* ### 安装Flannel

```shell
root@Slave:/home/zdyfjh# cd ./download/
root@Slave:/home/zdyfjh/download# ls
etcd-v3.3.10-linux-amd64         flannel-v0.9.0-linux-amd64.tar.gz  kubernetes-server-linux-amd64.tar.gz
etcd-v3.3.10-linux-amd64.tar.gz  kubernetes
root@Slave:/home/zdyfjh/download# tar -zxvf flannel-v0.9.0-linux-amd64.tar.gz 
flanneld
mk-docker-opts.sh
README.md
root@Slave:/home/zdyfjh/download# mv flanneld /usr/local/bin/
root@Slave:/home/zdyfjh/download# mv mk-docker-opts.sh /usr/local/bin/
root@Slave:/home/zdyfjh/download# vim /etc/systemd/system/flanneld.service
root@Slave:/home/zdyfjh/download# cat /etc/systemd/system/flanneld.service 
[Unit]
Description=Flanneld
Documentation=https://github.com/coreos/flannel
After=network.target
After=network-online.target
Wants=network-online.target
After=etcd.service
Before=docker.service
[Service]
User=root
ExecStart=/usr/local/bin/flanneld \
        -etcd-endpoints="https://192.168.10.10:2379,https://192.168.10.11:2379,https://192.168.10.12:2379,https://192.168.10.13:2379,https://192.168.10.14:2379,https://192.168.10.15:2379" \
        -ip-masq=true \
        -etcd-cafile=/etc/kubernetes/ssl/ca.pem \
        -etcd-certfile=/etc/kubernetes/ssl/etcd.pem \
        -etcd-keyfile=/etc/kubernetes/ssl/etcd-key.pem \
    -iface=eno4 \
        -etcd-prefix=/flannel/network
ExecStartPost=/usr/local/bin/mk-docker-opts.sh -d /etc/default/docker -c #注意最后的/etc/default/docker需要将其中的内容拷贝到docker.service中
Restart=on-failure
Type=notify
LimitNOFILE=65536

[Install]
WantedBy=multi-user.target
root@Slave:/etc/default# cat /etc/default/docker 
DOCKER_OPTS=" --bip=10.254.69.1/24 --ip-masq=false --mtu=1450"
root@Slave:/etc/default# vim /lib/systemd/system/docker.service 
root@Slave:/etc/default# cat /lib/systemd/system/docker.service 
...
ExecStart=/usr/bin/dockerd -H fd:// --containerd=/run/containerd/containerd.sock --bip=10.254.69.1/24 --ip-masq=false --mtu=1450 #在末尾加上/etc/default/docker中的启动内容
...
root@Slave:/home/zdyfjh/download# systemctl daemon-reload
root@Slave:/home/zdyfjh/download# systemctl enable flanneld
Created symlink from /etc/systemd/system/multi-user.target.wants/flanneld.service to /etc/systemd/system/flanneld.service.
root@Slave:/home/zdyfjh/download# systemctl start flanneld
root@Slave:/home/zdyfjh/download# systemctl restart docker
root@Slave:/home/zdyfjh/download# systemctl restart kubelet
root@Slave:/home/zdyfjh/download# systemctl restart kube-proxy
root@Slave:/home/zdyfjh/download# systemctl enable kubelet
root@Slave:/home/zdyfjh/download# systemctl enable kube-proxy
root@Slave3:/home/zdyfjh# iptables -P INPUT ACCEPT
root@Slave3:/home/zdyfjh# iptables -P FORWARD ACCEPT
root@Slave3:/home/zdyfjh# iptables -F
root@Slave3:/home/zdyfjh# iptables -L -n
```

* ### 验证flannel

要验证flannel的跨主机网络通信能力，最开始需要验证flannel是否添加了网络设备并更改了docker0网桥的网段。

```shell
root@Master:/home/zdyfjh/download# ifconfig 

...

docker0   Link encap:Ethernet  HWaddr 56:84:7a:fe:97:99  
          inet addr:10.254.99.1  Bcast:0.0.0.0  Mask:255.255.255.0
          UP BROADCAST MULTICAST  MTU:1500  Metric:1
          RX packets:0 errors:0 dropped:0 overruns:0 frame:0
          TX packets:0 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:0 (0.0 B)  TX bytes:0 (0.0 B)

flannel.1 Link encap:Ethernet  HWaddr 46:a2:b1:8c:08:94  
          inet addr:10.254.99.0  Bcast:0.0.0.0  Mask:255.255.255.255
          UP BROADCAST RUNNING MULTICAST  MTU:1450  Metric:1
          RX packets:3 errors:0 dropped:0 overruns:0 frame:0
          TX packets:3 errors:0 dropped:9 overruns:0 carrier:0
          collisions:0 txqueuelen:0 
          RX bytes:252 (252.0 B)  TX bytes:252 (252.0 B)

...

```

可以看到：docker0网桥的网段已经更改，而且已经添加了flannel.1网络设备，说明配置安装工作已经完成，接下来验证flannel的跨主机通信能力，该验证通过建立多个跨主机容器相互ping实验来验证。

```shell
root@Master:/home/zdyfjh/download# vim test.yaml 
root@Master:/home/zdyfjh/download# cat test.yaml 
apiVersion: extensions/v1beta1 
kind: Deployment 
metadata: 
  name: tomcat-dm
spec: 
  replicas: 6
  template: 
    metadata: 
      labels: 
        name: tomcat
    spec: 
      containers: 
        - name: tomcat
          image: tomcat 
          imagePullPolicy: IfNotPresent
          ports: 
            - containerPort: 80
root@Master:/home/zdyfjh/download# kubectl create -f test.yaml
root@Master:/home/zdyfjh/download# kubectl get po -o wide
NAME                         READY     STATUS    RESTARTS   AGE       IP             NODE
tomcat-dm-6bbbfd6b66-2p2dz   1/1       Running   0          8m        10.254.106.2   slave1
tomcat-dm-6bbbfd6b66-bx5x8   1/1       Running   0          8m        10.254.70.2    slave3
tomcat-dm-6bbbfd6b66-c7lsx   1/1       Running   0          8m        10.254.88.2    slave4
tomcat-dm-6bbbfd6b66-g8jqd   1/1       Running   0          8m        10.254.125.2   slave2
tomcat-dm-6bbbfd6b66-kqwgl   1/1       Running   0          8m        10.254.111.2   slave5
tomcat-dm-6bbbfd6b66-tpl4t   1/1       Running   0          8m        10.254.70.3    slave3
root@Master:/home/zdyfjh/download# ping 10.254.106.2
PING 10.254.106.2 (10.254.106.2) 56(84) bytes of data.
64 bytes from 10.254.106.2: icmp_seq=1 ttl=63 time=0.343 ms
64 bytes from 10.254.106.2: icmp_seq=2 ttl=63 time=0.254 ms
64 bytes from 10.254.106.2: icmp_seq=3 ttl=63 time=0.253 ms
^C
--- 10.254.106.2 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2028ms
rtt min/avg/max/mdev = 0.253/0.283/0.343/0.044 ms
```

# 安装CoreDNS解析插件

## Master节点上的操作

* ### 下载coredns镜像，配置dns并启动

```shell
root@Master:/home/zdyfjh/download# docker pull coredns/coredns:1.0.4
1.0.4: Pulling from coredns/coredns
3690ec4760f9: Pull complete 
0e610ed088f8: Pull complete 
dabd0d361ae6: Pull complete 
Digest: sha256:d291f8b87eab26845a0c4605df4194924806712c4f624b9a9ddfc9d382b3ddbd
Status: Downloaded newer image for coredns/coredns:1.0.4
root@Master:/home/zdyfjh/download# vim coredns.yaml 
root@Master:/home/zdyfjh/download# cat coredns.yaml 
apiVersion: v1
kind: ServiceAccount
metadata:
  name: coredns
  namespace: kube-system
---
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRole
metadata:
  labels:
    kubernetes.io/bootstrapping: rbac-defaults
  name: system:coredns
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  - services
  - pods
  - namespaces
  verbs:
  - list
  - watch
- apiGroups:
  - ""
  resources:
  - nodes
  verbs:
  - get
---
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRoleBinding
metadata:
  annotations:
    rbac.authorization.kubernetes.io/autoupdate: "true"
  labels:
    kubernetes.io/bootstrapping: rbac-defaults
  name: system:coredns
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: system:coredns
subjects:
- kind: ServiceAccount
  name: coredns
  namespace: kube-system
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: coredns
  namespace: kube-system
data:
  Corefile: |
    .:53 {
        errors
        health
        kubernetes cluster.local 10.254.0.0/18 {
          pods insecure
          upstream /etc/resolv.conf #这里要改成机器上dns解析的conf文件地址
          fallthrough in-addr.arpa ip6.arpa
        }
        prometheus :9153
        proxy . /etc/resolv.conf
        cache 30
        loadbalance
    }
---
apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: coredns
  namespace: kube-system
  labels:
    k8s-app: kube-dns
    kubernetes.io/name: "CoreDNS"
spec:
  replicas: 2
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxUnavailable: 1
  selector:
    matchLabels:
      k8s-app: kube-dns
  template:
    metadata:
      labels:
        k8s-app: kube-dns
    spec:
      serviceAccountName: coredns
      tolerations:
        - key: node-role.kubernetes.io/master
          effect: NoSchedule
        - key: "CriticalAddonsOnly"
          operator: "Exists"
      containers:
      - name: coredns
        image: coredns/coredns:1.0.4
        imagePullPolicy: IfNotPresent
        resources:
          limits:
            memory: 170Mi
          requests:
            cpu: 100m
            memory: 70Mi
        args: [ "-conf", "/etc/coredns/Corefile" ]
        volumeMounts:
        - name: config-volume
          mountPath: /etc/coredns
          readOnly: true
        ports:
        - containerPort: 53
          name: dns
          protocol: UDP
        - containerPort: 53
          name: dns-tcp
          protocol: TCP
        - containerPort: 9153
          name: metrics
          protocol: TCP
        securityContext:
          allowPrivilegeEscalation: false
          capabilities:
            add:
            - NET_BIND_SERVICE
            drop:
            - all
          readOnlyRootFilesystem: true
        livenessProbe:
          httpGet:
            path: /health
            port: 8080
            scheme: HTTP
          initialDelaySeconds: 60
          timeoutSeconds: 5
          successThreshold: 1
          failureThreshold: 5
      dnsPolicy: Default
      volumes:
        - name: config-volume
          configMap:
            name: coredns
            items:
            - key: Corefile
              path: Corefile
---
apiVersion: v1
kind: Service
metadata:
  name: kube-dns
  namespace: kube-system
  annotations:
    prometheus.io/port: "9153"
    prometheus.io/scrape: "true"
  labels:
    k8s-app: kube-dns
    kubernetes.io/cluster-service: "true"
    kubernetes.io/name: "CoreDNS"
spec:
  selector:
    k8s-app: kube-dns
  clusterIP: 10.254.0.2
  ports:
  - name: dns
    port: 53
    protocol: UDP
  - name: dns-tcp
    port: 53
    protocol: TCP
root@Master:/home/zdyfjh/download# kubectl apply -f coredns.yaml
serviceaccount "coredns" created
clusterrole "system:coredns" created
clusterrolebinding "system:coredns" created
configmap "coredns" created
deployment "coredns" created
service "kube-dns" created
root@Master:/home/zdyfjh/download# kubectl get po -o wide -n kube-system
NAME                       READY     STATUS    RESTARTS   AGE       IP             NODE
coredns-5955c6bfd4-6fs8l   1/1       Running   0          32s       10.254.125.2   slave2
coredns-5955c6bfd4-g7nlr   1/1       Running   0          32s       10.254.88.2    slave4
root@Master:/home/zdyfjh/download# kubectl get pod,svc -n kube-system -o wide
NAME                          READY     STATUS    RESTARTS   AGE       IP             NODE
po/coredns-5955c6bfd4-6fs8l   1/1       Running   0          50s       10.254.125.2   slave2
po/coredns-5955c6bfd4-g7nlr   1/1       Running   0          50s       10.254.88.2    slave4

NAME           TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)         AGE       SELECTOR
svc/kube-dns   ClusterIP   10.254.0.2   <none>        53/UDP,53/TCP   50s       k8s-app=kube-dns
```

* ### 测试coreDNS

```shell
root@Master:/home/zdyfjh/download# kubectl create -f test.yaml 
deployment "tomcat-dm" created
root@Master:/home/zdyfjh/download# kubectl get po -o wide
NAME                         READY     STATUS    RESTARTS   AGE       IP             NODE
tomcat-dm-6bbbfd6b66-82gvr   1/1       Running   0          5s        10.254.88.3    slave4
tomcat-dm-6bbbfd6b66-8jz54   1/1       Running   0          5s        10.254.111.3   slave5
tomcat-dm-6bbbfd6b66-bbh4d   1/1       Running   0          5s        10.254.70.2    slave3
tomcat-dm-6bbbfd6b66-f7bwn   1/1       Running   0          5s        10.254.111.2   slave5
tomcat-dm-6bbbfd6b66-p7lz4   1/1       Running   0          5s        10.254.106.2   slave1
tomcat-dm-6bbbfd6b66-xv67k   1/1       Running   0          5s        10.254.125.3   slave2
root@Master:/home/zdyfjh/download# kubectl exec -it tomcat-dm-6bbbfd6b66-bbh4d /bin/bash
root@tomcat-dm-6bbbfd6b66-bbh4d:/usr/local/tomcat# ping baidu.com
PING baidu.com (123.125.115.110) 56(84) bytes of data.
64 bytes from 123.125.115.110 (123.125.115.110): icmp_seq=1 ttl=49 time=33.7 ms
64 bytes from 123.125.115.110 (123.125.115.110): icmp_seq=2 ttl=49 time=33.7 ms
64 bytes from 123.125.115.110 (123.125.115.110): icmp_seq=3 ttl=49 time=33.6 ms
^C
--- baidu.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2003ms
rtt min/avg/max/mdev = 33.680/33.731/33.788/0.216 ms
```

如上面代码所示，`baidu.com`已经`ping`通，说明容器内部的域名解析服务是正常工作的。

# 后记

算了，没后记了，希望以后看到这篇文档安装系统的小伙伴们能一次过。

2018-10-24

# Ref

[基于二进制安装 kubernetes 1.9.6](https://blog.csdn.net/qq_21816375/article/details/80288937)